###  Stronicowanie wyników z dwóch tabel przy pomocy frazy kluczowej IN

Dzięki temu optymalizuje to działanie aplikacji i zapytań do bazy danych.


W kontrolerze dodajemy metodę, która pozwoli pobrać listę postów wraz z komentarzami.

In [ ]:
@GetMapping("/posts/comments")
public List<Post> getPostsWithComments(@RequestParam(required = false) int page) {
    int pageNumber = page >= 0 ? page : 0;
    return postService.getPostsWithComments(pageNumber);
    
}

Żeby pobrać komentarze do powyższej listy postów, musimy stworzyć repository CommentRepository

In [ ]:
@Repository
public interface CommentRepository extends CrudRepository<Comment, Long> {

    @Query  # is this query necessary? I think it's not because we use "method name queries".
    List<Comment> findAllByPostIdIn(List<Long> ids);
}

Wstrzykujemy CommentRepository do serwisu

In [ ]:
private final CommentRepository commentRepository;

## and we add it to constructor

W serwisie PostService dodajemy metodę getPostsWithComments, w której wyciągamy listę Postów, oraz listę "id" (w tym przypadku jest to typ Long czyli "List< Long>")

In [ ]:
public List<Post> getPostsWithComments(int page) {
    
    # 1. Get all posts from required page
    # 2. Make a list of all ids from the list of all posts
    # 3. Make a list of all comments fin
    
    # getting list of all posts
    List<Post> allPosts = postRespository.findAllPosts(PageRequest.of(page, PAGE_SIZE));
    # getting list of all posts ids
    List<Long> ids = allPosts.stream()
            .map(Post::getId)
            .collect(Collectors.toList());
    # we pass as a parameter the list of ids to commentRepository
    List<Comment> comments = commentRepository.findAllByPostIdIn(ids)
    
    # now we have to connect comments with correct posts
    allPosts.forEach(post -> post.setComment(extractComments(comments, post.getId()))); # extractComments method must be created
    return allPosts;
}

In [ ]:
private List<Comment> extractComment(List<Comment> comments, long id) {
    return comments.stream()
            .filter(comment -> comment.getPostId() == id) ## if we don't have this field in model we have to add it!
            .collect(Collectors.toList());
    ## also in the Post entity we have to change from @JoinColumn(name = "post_id") to (name = "postId")!!!!!!!!!!!!
}